In [1]:
import json
import numpy as np

Load the CIFAR-10 structures testing accuracy, valid accuracy, parameter sizes.

In [2]:
with open('../data/all_structure_accuracy.json') as tt:
    structure = json.load(tt)
structure_dict = {}
for s,acc in structure:
    structure_dict[str(s)] = acc
with open('../data/all_structure_valid_accuracy.json') as tt:
    structure = json.load(tt)
structure_dict_valid = {}
for s,acc in structure:
    structure_dict_valid[str(s)] = acc
with open('../data/all_structure_params.json') as tt:
    structure = json.load(tt)
params_dict = {}
for s,acc in structure:
    params_dict[str(s)] = acc

Some functions.

In [3]:
NAS_BENCH_201         = ['none', 'skip_connect', 'nor_conv_1x1', 'nor_conv_3x3', 'avg_pool_3x3']
def generate_arch(structure):
    NAS_BENCH_201 = ['none', 'skip_connect', 'nor_conv_1x1', 'nor_conv_3x3', 'avg_pool_3x3']
    strings = []
    for i in range(3):
        
        string = '|'.join([NAS_BENCH_201[structure[i][k]]+'~{:}'.format(k) for k in range(i+1)])
        string = '|{:}|'.format(string)
        strings.append( string )
    return '+'.join(strings)

Selection.

In [4]:
names = ['record_cifar10_17_16_80000_0_256_64_4_1_50_0.5_0.0_']
steps = [50]
max_params = 1.5315459999999999
alpha = 1.4
beta = 5.2
T = 10000
Total = 80000
randids = ['111','222','333','444','555']
for name,st in zip(names,steps):
    corr = []
    params_loss_sets = []
    for randid in randids:
        estimat_dict = {}
        params_loss_set = []
        for idxxx in range((Total-T)//st,Total//st):
            with open('../archive/nasbench_results/'+name+randid+ '/%06d-ep.txt'%((idxxx+1)*st),'r') as tt:
                history = json.load(tt)
            sorted_history = sorted(history,key=lambda x: \
                              x[1] if x[2] >= 1 else 100, reverse=False)
            
            
            for structure,loss,count in sorted_history[:1]:
                arch = generate_arch(structure)
                cos = params_dict[str(structure)]
                if arch not in estimat_dict:
                    estimat_dict[arch] = \
                {'loss_set':[loss],'count':1,'number_rep':structure,'params':cos}
                else:
                    if estimat_dict[arch]['loss_set'][-1] != loss:
                        estimat_dict[arch]['loss_set'].append(loss)
                        estimat_dict[arch]['count'] += 1
        
        final_acc_set = []
        for arch in estimat_dict:
            if estimat_dict[arch]['count'] >= 1:
                loss = estimat_dict[arch]['loss_set'][-1]
                loss = loss + alpha * np.exp(-beta * estimat_dict[arch]['params']/max_params) 
                final_acc_set.append([loss,estimat_dict[arch]['number_rep']])
                params_loss_set.append([estimat_dict[arch]['params'],loss,estimat_dict[arch]['number_rep']])
        params_loss_set.sort()
        params_loss_sets.append(params_loss_set)
        final_acc_set.sort()
        new_accuracy_set = []
    
    top_acc1s_valid = []
    top_acc1s = []
    for k,randid in enumerate(randids):
        top_acc1 = []
        lowest_p = 100
        bests = []
        best_pareto = 0
        for indx,params in enumerate(params_loss_sets[k]):
            if params[1] <= lowest_p:
                lowest_p = params[1] 
                bests.append(params)
        new_acc_valid = structure_dict_valid[str(bests[-1][2])]
        new_acc = structure_dict[str(bests[-1][2])]
        top_acc1s_valid.append(new_acc_valid)
        top_acc1s.append(new_acc)
    print('Valid accuracy:',top_acc1s_valid)
    print('Test accuracy:',top_acc1s)

Valid accuracy: [91.5533333235677, 91.42266665120444, 91.49599997884116, 91.42266665120444, 91.60666665039064]
Test accuracy: [94.36333333333334, 94.29333333333334, 94.37333333333333, 94.29333333333334, 94.37333333333333]
